In [5]:
import optuna

ModuleNotFoundError: ignored

In [1]:
import xgboost

In [2]:
!pip  list

Package                       Version
----------------------------- ---------------------
absl-py                       1.4.0
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
array-record                  0.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
attrs                         23.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branca                        0.6.0
build                         0.10.0
CacheControl                  0.12.11
ca

In [3]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.7 MB/s eta 0:00:00


In [5]:
import optuna
import xgboost as xg
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [7]:
df = pd.read_csv("https://raw.githubusercontent.com/sunnysavita10/svm_kernel/main/Admission_Prediction.csv")

In [8]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [9]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          485 non-null    float64
 2   TOEFL Score        490 non-null    float64
 3   University Rating  485 non-null    float64
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 35.3 KB


In [11]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [15]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].median())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].median())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].median())

In [16]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [21]:
df = df.drop(['Serial No.'], axis = 1)

In [22]:
X= df.drop(['Chance of Admit'], axis = 1)

In [23]:
X.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,317.0,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0


In [24]:
y = df['Chance of Admit']

In [26]:
y.sample(5)

305    0.74
156    0.70
277    0.70
357    0.68
86     0.72
Name: Chance of Admit, dtype: float64

In [28]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   GRE Score          500 non-null    float64
 1   TOEFL Score        500 non-null    float64
 2   University Rating  500 non-null    float64
 3   SOP                500 non-null    float64
 4   LOR                500 non-null    float64
 5   CGPA               500 non-null    float64
 6   Research           500 non-null    int64  
dtypes: float64(6), int64(1)
memory usage: 27.5 KB


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 26)

In [31]:
X_train.shape

(350, 7)

In [32]:
std_sca = StandardScaler()

In [33]:
X_train = std_sca.fit_transform(X_train)

In [34]:
X_train.shape

(350, 7)

In [35]:
X_test = std_sca.fit_transform(X_test)

In [36]:
!nvidia-smi

Fri Jun  9 05:11:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [60]:
def objective(trail, data =X, target = y):
  train_x, test_x, train_y, test_y = train_test_split(data, target, test_size= 0.3, random_state = 1)
  param = {
      'tree_method':'gpu_hist',
      'reg_lamda':trail.suggest_loguniform('reg_lamda',1e-4,10.0),
      'reg_alpha':trail.suggest_loguniform('reg_alpha',1e-4,10.0),
      'colsample_bytree':trail.suggest_categorical('colsample_bytree',[.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
      'subsample': trail.suggest_categorical('subsample',[.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
      'learning_rate':trail.suggest_categorical('learning_rate',[0.00001,0.0003,0.008,0.02,0.1,1,8]),
      'n_estimator':300,
      'max_depth':trail.suggest_categorical('max_depth',[3,4,5,6,7,8,9,10,11,12]),
      'random_state':trail.suggest_categorical('random_state',[10,20,30,547,98547,1598]),
      'min_child_weight':trail.suggest_int('min_child_weight',1,200)

  }
  Xgb_reg_model = xg.XGBRegressor(**param)
  Xgb_reg_model.fit(train_x, train_y, eval_set = [(test_x, test_y)], verbose = True)
  pred = Xgb_reg_model.predict(test_x)
  mse = mean_squared_error(test_y, pred)
  return mse

In [61]:
find_params = optuna.create_study()
find_params.optimize(objective, n_trials = 15)
find_params.best_trial.params

[I 2023-06-09 05:58:41,286] A new study created in memory with name: no-name-a8a273cf-eecc-4cc5-b5b0-b63851c59a2e


[05:58:41] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimator", "reg_lamda" } are not used.

[0]	validation_0-rmse:0.25603
[1]	validation_0-rmse:0.25129
[2]	validation_0-rmse:0.24665
[3]	validation_0-rmse:0.24210
[4]	validation_0-rmse:0.23789
[5]	validation_0-rmse:0.23372
[6]	validation_0-rmse:0.22943
[7]	validation_0-rmse:0.22544
[8]	validation_0-rmse:0.22129
[9]	validation_0-rmse:0.21728
[10]	validation_0-rmse:0.21365
[11]	validation_0-rmse:0.20984
[12]	validation_0-rmse:0.20604
[13]	validation_0-rmse:0.20254
[14]	validation_0-rmse:0.19897
[15]	validation_0-rmse:0.19544
[16]	validation_0-rmse:0.19198
[17]	validation_0-rmse:0.18877
[18]	validation_0-rmse:0.18550
[19]	validation_0-rmse:0.18228
[20]	validation_0-rmse:0.17917
[21]	validation_0-rmse:0.17611
[22]	validation_0-rmse:0.17310
[23]	validation_0-rmse:0.17017
[24]	validation_0-rmse:0.16734
[25]	validation_0-rmse:0.16457
[26]	validation_0-rmse:0.16197
[27]	validation_0-rmse:0.15931
[28]	validation_0-rmse:0.15669
[29]	vali

<ipython-input-60-a6948f765ace>:5: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

<ipython-input-60-a6948f765ace>:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[38]	validation_0-rmse:0.13421
[39]	validation_0-rmse:0.13222
[40]	validation_0-rmse:0.13032
[41]	validation_0-rmse:0.12844
[42]	validation_0-rmse:0.12660
[43]	validation_0-rmse:0.12483
[44]	validation_0-rmse:0.12305
[45]	validation_0-rmse:0.12143
[46]	validation_0-rmse:0.11973
[47]	validation_0-rmse:0.11814
[48]	validation_0-rmse:0.11658
[49]	validation_0-rmse:0.11504
[50]	validation_0-rmse:0.11354
[51]	validation_0-rmse:0.11209
[52]	validation_0-rmse:0.11071
[53]	validation_0-rmse:0.10937
[54]	validation_0-rmse:0.10811
[55]	validation_0-rmse:0.10678
[56]	validation_0-rmse:0.10557
[57]	validation_0-rmse:0.10439
[58]	validation_0-rmse:0.10320
[59]	validation_0-rmse:0.10206
[60]	validation_0-rmse:0.10095
[61]	validation_0-rmse:0.09988
[62]	validation_0-rmse:0.09886
[63]	validation_0-rmse:0.09788
[64]	validation_0-rmse:0.09690
[65]	validation_0-rmse:0.09590
[66]	validation_0-rmse:0.09492
[67]	validation_0-rmse:0.09406
[68]	validation_0-rmse:0.09318
[69]	validation_0-rmse:0.09233
[70]	val

[I 2023-06-09 05:58:41,812] Trial 0 finished with value: 0.005845570747643798 and parameters: {'reg_lamda': 4.702796495937023, 'reg_alpha': 0.023609117416877465, 'colsample_bytree': 0.8, 'subsample': 1, 'learning_rate': 0.02, 'max_depth': 8, 'random_state': 1598, 'min_child_weight': 4}. Best is trial 0 with value: 0.005845570747643798.


...

[05:58:47] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimator", "reg_lamda" } are not used.

[0]	validation_0-rmse:0.23785
[1]	validation_0-rmse:0.21843
[2]	validation_0-rmse:0.19997
[3]	validation_0-rmse:0.18534
[4]	validation_0-rmse:0.17161
[5]	validation_0-rmse:0.16034
[6]	validation_0-rmse:0.15027
[7]	validation_0-rmse:0.13964
[8]	validation_0-rmse:0.13013
[9]	validation_0-rmse:0.12172
[10]	validation_0-rmse:0.11448
[11]	validation_0-rmse:0.10873
[12]	validation_0-rmse:0.10324
[13]	validation_0-rmse:0.09923
[14]	validation_0-rmse:0.09557
[15]	validation_0-rmse:0.09178
[16]	validation_0-rmse:0.08882
[17]	validation_0-rmse:0.08671
[18]	validation_0-rmse:0.08446
[19]	validation_0-rmse:0.08263
[20]	validation_0-rmse:0.08159
[21]	validation_0-rmse:0.08051
[22]	validation_0-rmse:0.07932
[23]	validation_0-rmse:0.07841
[24]	validation_0-rmse:0.07816
[25]	validation_0-rmse:0.07769
[26]	validation_0-rmse:0.07724
[27]	validation_0-rmse:0.07694
[28]	validation_0-rmse:0.07692


<ipython-input-60-a6948f765ace>:5: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

<ipython-input-60-a6948f765ace>:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.



[29]	validation_0-rmse:0.07669
[30]	validation_0-rmse:0.07636
[31]	validation_0-rmse:0.07625
[32]	validation_0-rmse:0.07599
[33]	validation_0-rmse:0.07588
[34]	validation_0-rmse:0.07572
[35]	validation_0-rmse:0.07574
[36]	validation_0-rmse:0.07551
[37]	validation_0-rmse:0.07548
[38]	validation_0-rmse:0.07551
[39]	validation_0-rmse:0.07528
[40]	validation_0-rmse:0.07513
[41]	validation_0-rmse:0.07499
[42]	validation_0-rmse:0.07472
[43]	validation_0-rmse:0.07462
[44]	validation_0-rmse:0.07464
[45]	validation_0-rmse:0.07465
[46]	validation_0-rmse:0.07465
[47]	validation_0-rmse:0.07465
[48]	validation_0-rmse:0.07457
[49]	validation_0-rmse:0.07457
[50]	validation_0-rmse:0.07448
[51]	validation_0-rmse:0.07446
[52]	validation_0-rmse:0.07447
[53]	validation_0-rmse:0.07438
[54]	validation_0-rmse:0.07421
[55]	validation_0-rmse:0.07419
[56]	validation_0-rmse:0.07378
[57]	validation_0-rmse:0.07378
[58]	validation_0-rmse:0.07351
[59]	validation_0-rmse:0.07351
[60]	validation_0-rmse:0.07352
[61]	val

[I 2023-06-09 05:58:47,965] Trial 14 finished with value: 0.005303820566870063 and parameters: {'reg_lamda': 0.5136285580702948, 'reg_alpha': 0.38326985967364174, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.1, 'max_depth': 6, 'random_state': 98547, 'min_child_weight': 46}. Best is trial 11 with value: 0.004887290110555546.


{'reg_lamda': 6.816455912628457,
 'reg_alpha': 0.22849249126804094,
 'colsample_bytree': 1,
 'subsample': 0.8,
 'learning_rate': 0.1,
 'max_depth': 6,
 'random_state': 98547,
 'min_child_weight': 1}

In [46]:
find_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_colsample_bytree,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_reg_alpha,params_reg_lamda,params_subsample,state
0,0,2.064562e-02,2023-06-09 05:35:48.905258,2023-06-09 05:35:49.336226,0 days 00:00:00.430968,0.1,0.00800,8,25,20,0.081311,3.146866,0.8,COMPLETE
1,1,6.804867e-02,2023-06-09 05:35:49.337998,2023-06-09 05:35:49.697347,0 days 00:00:00.359349,0.6,1.00000,10,124,98547,0.000159,0.007569,0.1,COMPLETE
2,2,6.793674e-02,2023-06-09 05:35:49.699181,2023-06-09 05:35:50.140702,0 days 00:00:00.441521,0.2,0.00001,3,1,30,0.003561,0.000100,0.3,COMPLETE
3,3,6.792673e-02,2023-06-09 05:35:50.142917,2023-06-09 05:35:50.569618,0 days 00:00:00.426701,0.8,0.00001,12,41,1598,0.221487,1.709180,0.9,COMPLETE
4,4,6.804867e-02,2023-06-09 05:35:50.574120,2023-06-09 05:35:50.942061,0 days 00:00:00.367941,0.5,0.00001,4,182,20,0.000493,0.008858,0.3,COMPLETE
5,5,6.804867e-02,2023-06-09 05:35:50.943755,2023-06-09 05:35:51.316356,0 days 00:00:00.372601,1.0,8.00000,11,152,30,8.794261,2.367067,0.2,COMPLETE
6,6,2.252360e-02,2023-06-09 05:35:51.320972,2023-06-09 05:35:51.697767,0 days 00:00:00.376795,0.2,0.10000,7,184,10,0.022485,0.453978,0.8,COMPLETE
7,7,5.933638e-02,2023-06-09 05:35:51.699473,2023-06-09 05:35:52.096862,0 days 00:00:00.397389,0.5,0.02000,8,121,98547,0.020939,0.046241,0.3,COMPLETE
8,8,6.540796e-02,2023-06-09 05:35:52.098884,2023-06-09 05:35:52.478822,0 days 00:00:00.379938,1.0,0.00030,10,126,20,0.002490,0.076852,0.4,COMPLETE
9,9,1.924266e+74,2023-06-09 05:35:52.481245,2023-06-09 05:35:52.839705,0 days 00:00:00.358460,0.6,8.00000,4,103,20,0.991831,0.540574,0.3,COMPLETE


In [48]:
optuna.visualization.plot_optimization_history(find_params)

In [49]:
optuna.visualization.plot_slice(find_params)

In [53]:
optuna.visualization.plot_contour(find_params, params =['reg_lamda', 'reg_alpha'])

In [62]:
best_params ={'reg_lamda': 6.816455912628457,
 'reg_alpha': 0.22849249126804094,
 'colsample_bytree': 1,
 'subsample': 0.8,
 'learning_rate': 0.1,
 'max_depth': 6,
 'random_state': 98547,
 'min_child_weight': 1}

In [63]:
model = xg.XGBRegressor(**best_params)

In [64]:
model.fit(X_train, y_train)

[05:59:13] WARNING: ../src/learner.cc:767: 
Parameters: { "reg_lamda" } are not used.



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=98547, ...)

In [65]:
pred = model.predict(X_test)

In [66]:
from sklearn.metrics import r2_score
r2_score(y_test, pred)

0.73623012371882

In [67]:
from sklearn.ensemble import RandomForestRegressor

In [68]:
model2 = RandomForestRegressor()

In [69]:
model2.fit(X_train, y_train)
pred2 = model2.predict(X_test)
r2_score(y_test, pred2)

0.7323962170537801

In [ ]:
you need to take one more data regression and explore more and more about this optuna
take classificaiton dataset and check with the accracy with different different hyperparameter(optuna)
write atleast 5 differences between optuna and grid search and random search